In [3]:
%reload_ext autoreload
%autoreload 2

import sys

sys.path.append('../Customized_Components') #this path is not right for this PC. Maybne this will work?
from bandaged_dolan import DolanJunctionBandage
from houcklab_qubit import DiffTransmonRounded

import warnings

import shapely
from CoupledLineTee import CoupledLineTee
from dolan_junction import DolanJunction
from LaunchpadWirebondCustom import LaunchpadWirebondCustom
from shapely.errors import ShapelyDeprecationWarning
from single_pad_transmon_pocket import TransmonPocket_Single

warnings.filterwarnings("ignore", category=ShapelyDeprecationWarning)
from collections import OrderedDict

import Default_res_params as dp
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import copy

# from qiskit_metal.qlibrary.couplers.coupled_line_tee import CoupledLineTee
# from qiskit_metal.qlibrary.terminations.launchpad_wb import LaunchpadWirebond
import pyEPR as epr
from qiskit_metal import Dict, Headings, MetalGUI, designs, draw
from qiskit_metal.qlibrary.terminations.open_to_ground import OpenToGround
from qiskit_metal.qlibrary.terminations.short_to_ground import ShortToGround
from qiskit_metal.qlibrary.tlines.anchored_path import RouteAnchors
from qiskit_metal.qlibrary.tlines.mixed_path import RouteMeander, RouteMixed
from qiskit_metal.qlibrary.tlines.straight_path import RouteStraight
from qiskit_metal.toolbox_metal.parsing import *


In [2]:
gui = MetalGUI(dp.design)
design = dp.design
design.overwrite_enabled = True

01:43PM 39s CRITICAL [_qt_message_handler]: line: 0, func: None(), file: None  WARNING: Populating font family aliases took 163 ms. Replace uses of missing font family "Courier" with one that exists to avoid this cost. 



2024-11-07 13:43:39.951 python[3309:1681169] +[IMKClient subclass]: chose IMKClient_Modern
2024-11-07 13:43:39.951 python[3309:1681169] +[IMKInputSession subclass]: chose IMKInputSession_Modern


In [7]:
default_options = Dict(
        cut_l='1000um',
        cut_h='1200um',
        gap='55um',
        w = '90um',
        l = '900um',
        r = '60um',
        cpw_l = '50um',
        coupling_gap = '30um',
        JJ_cutout_w = '70um',
        JJ_cutout_h = '70um',
        JJ_cutout_r = '150um',
        JJ_c_contact_l = '40um',
        JJ_c_contact_r = '2.5um',
        JJ_c_contact_w = '10um',
        coupling_d = '200um',
        coupling_pad_w = '90um',
        coupling_r = '40um',
        cpw_pin = '10um',
        chip='main',
        resolution = '10',
        junction = 'False',
        orientation = '0',
        JJ_c_contact_shortl = '10um',
        istunnel = 'False')

## Calculate Qubit Params

In [8]:
import Transmon_property as tp

In [10]:
cap = 80*dp.u.fF
fq, (w1,alpha) = tp.freq_from_cap(cap)

In [13]:
fq.to(dp.u.GHz)

<Quantity 4.69305741 GHz>

In [14]:
alpha.to(dp.u.MHz)

<Quantity 242.12786656 MHz>

In [6]:
import scqubits as scq
import numpy as np
import matplotlib.pyplot as plt
import astropy.units as u
import astropy.constants as c

# Define the transmon parameters
C = 60 * u.fF  # Capacitance in femtofarads
L = 13 * u.nH  # Inductance in nanohenrys
# Calculate the flux quantum Phi0
Phi0 = (c.h / (2 * c.e.si))  # Flux quantum in Weber

# Calculate the Josephson energy EJ and charging energy EC
EJ = (((Phi0/2/np.pi)**2/L).to(u.J)/c.h).to(u.GHz)#, equivalencies=u.spectral())  # Convert to GHz
EC = (c.e.si**2 / (2 * C)/c.h).to(u.GHz)
ratio = 70
def calculate_qubit_freq(L,ratio = ratio):
    
    EJ = (((Phi0/2/np.pi)**2/L).to(u.J)/c.h).to(u.GHz)
    #, equivalencies=u.spectral())  # Convert to GHz
    EC = EJ/ratio
    ng = 0.0   # Offset charge
    nlev = 10  # Number of energy levels to consider

    # Print calculated EJ and EC
    # print(f"Calculated EJ (GHz): {EJ}")
    # print(f"Calculated EC (GHz): {EC}")

    # Create a Transmon object
    transmon = scq.Transmon(EJ=EJ.value, EC=EC.value, ng=ng, ncut=30, truncated_dim=nlev)

    # Calculate the eigenenergies
    eigenenergies = transmon.eigenvals()

    # Calculate the transition frequencies (in GHz)
    transition_frequencies = np.diff(eigenenergies)

    # Calculate the anharmonicity (in GHz)
    anharmonicity = transition_frequencies[1] - transition_frequencies[0]

# Print the results
    # print(f"Transition frequencies (GHz): {transition_frequencies}")
    # print(f"Anharmonicity (GHz): {anharmonicity}")
    # print('Ej/Ec', EJ/EC)
    return transition_frequencies[0]#EJ,anharmonicity

# Plot the energy levels
# plt.figure(figsize=(8, 6))
# plt.plot(eigenenergies, 'bo-', label='Eigenenergies')
# plt.xlabel('Energy Level Index')
# plt.ylabel('Energy (GHz)')
# plt.title('Transmon Eigenenergies')
# plt.legend()
# plt.grid(True)
# plt.show()

In [133]:
calculate_qubit_freq(10*u.nH)

5.281322018983346

In [134]:
target_function(10,5)

0.28132201898334586

In [135]:
from scipy.optimize import root_scalar

# Define the calculate_qubit_frequency function

def target_function(L, desired_frequency):
    L = L * u.nH  # Convert L to nanohenrys
    calculated_frequency = calculate_qubit_freq(L)
    return calculated_frequency - desired_frequency



In [136]:
# Desired qubit frequency
desired_frequency = 3.5   # Desired frequency in GHz

# Use scipy.optimize.root_scalar to find the root
result = root_scalar(target_function, args=(desired_frequency,), bracket=[1, 100], method='brentq')

# Print the result
if result.converged:
    L_optimal = result.root
    print(f"Optimal inductance L: {L_optimal} nH")
else:
    print("Root finding did not converge")

Optimal inductance L: 15.08949148280957 nH


In [137]:
def inductance_to_cap(L,ratio = ratio):
    EJ = (((Phi0/2/np.pi)**2/L).to(u.J)/c.h).to(u.GHz)
    EC = EJ/ratio
    return (c.e.si**2 / (2 * EC)/c.h).to(u.fF)

In [138]:
inductance_to_cap(L_optimal*u.nH)

<Quantity 125.16759069 fF>

In [139]:
result_L = []
result_cap = []
for desired_frequency in np.linspace(3,6,100):
    result = root_scalar(target_function, args=(desired_frequency,), bracket=[1, 100], method='brentq')

    # Print the result
    if result.converged:
        L_optimal = result.root
        result_L += [L_optimal]
        result_cap += [inductance_to_cap(L_optimal*u.nH).to(u.fF).value]
        # print(f"Optimal inductance L: {L_optimal} nH")
    else:
        print("Root finding did not converge")

In [140]:
import matplotlib.pyplot as plt
import numpy as np

# Example data
x = np.linspace(3,6,100)
y1 = result_L
y2 = result_cap

# Create the figure and the first y-axis
fig, ax1 = plt.subplots()

# Plot the first variable
ax1.plot(x, y1, 'b-', label='sin(x)')
ax1.set_xlabel('Target Frequency (GHz)')
ax1.set_ylabel('Inductance Value (nH)', color='b')
ax1.tick_params(axis='y', labelcolor='b')

# Create the second y-axis and plot the second variable
ax2 = ax1.twinx()
ax2.plot(x, y2, 'r--', label='cos(x)')
ax2.set_ylabel('Capacitance (fF)', color='r')
ax2.tick_params(axis='y', labelcolor='r')

# Add legends
ax1.legend(loc='upper left')
ax2.legend(loc='upper right')

# Add grid and title
ax1.grid(True)
plt.title('Plot with Two Different Y-Axes')

# Show the plot
plt.show()

In [141]:
from Transmon_specifications import find_junction_area, find_junction_capacitance

In [142]:
def pad_len_from_C(c):
    slope = (900-500)/(101.367-61.365)
    intercept = 500 - slope*61.365
    
    return slope*c.to(u.fF).value + intercept 
def unitless_find_j_area(L):
    return find_junction_area(L*u.nH,Jc = 0.42*u.uA/u.um**2).to(u.um**2).value

def unitless_find_pad_len(C):
    slope = (900-500)/(101.367-61.365)
    intercept = 500 - slope*61.365
    
    return slope*C + intercept

In [131]:
import matplotlib.pyplot as plt
import numpy as np

# Example data
x = np.linspace(3,6,100)
y1 = np.array(result_L)
y2 = np.array(result_cap)

j_size = unitless_find_j_area(y1)
c_size = unitless_find_pad_len(y2)

# Create the figure and the first y-axis
fig, ax1 = plt.subplots()

# Plot the first variable
ax1.plot(x, j_size, 'b-', label='sin(x)')
ax1.set_xlabel('Target Frequency (GHz)')
ax1.set_ylabel('Junction Size (um2)', color='b')
ax1.tick_params(axis='y', labelcolor='b')

# Create the second y-axis and plot the second variable
ax2 = ax1.twinx()
ax2.plot(x, c_size, 'r--', label='cos(x)')
ax2.set_ylabel('Pad length (um)', color='r')
ax2.tick_params(axis='y', labelcolor='r')

# Add legends
ax1.legend(loc='upper left')
ax2.legend(loc='upper right')

# Add grid and title
ax1.grid(True)
plt.title('Ej/Ec = 65')
plt.savefig('Ej_Ec_65.png')
# Show the plot
plt.show()

In [114]:
find_junction_area(10*u.nH, Jc = 0.42*u.uA/u.um**2)

<Quantity 0.07835857 um2>

In [118]:
pad_len_from_C(80*u.fF)

19.362899999999996

In [ ]:
def convert_capacitance_to_energy(capacitance):
    C = capacitance*u.fF
    return (c.e.si**2 / (2 * C)/c.h).to(u.GHz)

def get_cap_params(caps, c_j):
    cap_matrix = copy.deepcopy(caps)
    T = np.array([[1,0,-1,0],
              [0,1,0,0],
              [1,0,1,0],
              [0,0,0,1]])
    cap_matrix[0,2] -= cj
    cap_matrix[2,0] -= cj
    cap_matrix[2,2] += cj
    cap_matrix[0,0] += cj
    differential_cap_matrix = np.linalg.inv(T).T@(cap_matrix)@np.linalg.inv(T)
    inverse = np.linalg.inv(differential_cap_matrix)
    qubit_capacitance = np.reciprocal(invverse)[0,0]
    coupling_capacitance = np.reciprocal(inverse)[0,1]
    Ec = convert_capacitance_to_energy(qubit_capacitance)
    g = 8*convert_capacitance_to_energy(coupling_capactiance).to(u.MHz)
    return [Ec,g,qubit_capacitance,coupling_capacitance]
    